In [1]:
# import re
# text = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
# # pattern = "@(w+)"
# match = re.search(r"@(\w+)", text)
# print(match)

In [2]:
#pip install --upgrade pip

In [3]:
# import re
# text = "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"
# # pattern = "@(w+)"
# match = re.sub(r"https?://\S+", "", text)
# print(match)

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize


df = pd.read_csv('sentiment1.6BillionDataser.csv', header=None, encoding='latin-1')


In [5]:
df.shape
# df.isnull()

(1600000, 6)

In [6]:
df = df.drop([1,2,3,4], axis=1)

In [7]:
df.nunique()

0          2
5    1581466
dtype: int64

In [8]:
sentences = df[5]
sentences

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: 5, Length: 1600000, dtype: object

In [9]:
def lower(sent):
    return sent.lower()

In [10]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [11]:

# def lema(wordList):
#     res=[]
#     print(wordList)
#     for word in wordList:
#        res.append(lemma.lemmatize(word,pos='v'))
#     return res

In [12]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

for i in range(len(sentences)):
    sentences[i] = lower(sentences[i])
    
    #remove usernames
    sentences[i] = re.sub(r"@(\w+)", "", sentences[i])
    
    #remove any urls
    sentences[i] = re.sub(r"https?://\S+", "", sentences[i])
    
    #remove lagging spaces in b/w
    sentences[i] = re.sub(r"\s{2,}", " ", sentences[i])
    
    #remove punctuations
    sentences[i] = remove_punctuation(sentences[i])
    
    #stopwords
    stop_words = set(stopwords.words("english"))
    
    #split into words
    word_tokens = word_tokenize(sentences[i])
    
    #remove stopwords
    words = [word for word in word_tokens if word not in stop_words]
    
    #lemmatize words    
    res=[]
    for word in words:
       res.append(lemma.lemmatize(word,pos='v'))
    words=res
    
    #join thw words to form the final sentence
    sentences[i] = " ".join(words)
    

In [13]:
sentences

0          awww thats bummer shoulda get david carr third...
1          upset cant update facebook texting might cry r...
2           dive many time ball manage save 50 rest go bound
3                            whole body feel itchy like fire
4                                     behave im mad cant see
                                 ...                        
1599995                           wake school best feel ever
1599996          thewdbcom cool hear old walt interview â «
1599997                       ready mojo makeover ask detail
1599998    happy 38th birthday boo alll time tupac amaru ...
1599999                                 happy charitytuesday
Name: 5, Length: 1600000, dtype: object

In [14]:
target = df[0]
target.shape

(1600000,)

In [15]:
sentences.to_list()
sentences.shape

(1600000,)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences, target, test_size=0.20, random_state=0, stratify=target)

In [17]:
X_train

1397919                                             yay tell
1513658                                   kick ass rock band
1221652    ive live pegasus flight path long think would ...
627968                                      link doesnt work
798122                                     awwww poor metria
                                 ...                        
343350                                  didnt go work excite
92845      turn say quotand thats mother day yepp best pa...
1307036    pixelated executioner move blogger commission ...
1402805           wow mom officially post 3 tweet clap mommy
1279660    wckd review tony good babethey normally always...
Name: 5, Length: 1280000, dtype: object

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = CountVectorizer(max_features = 88)
tfidf_mat = vectorizer.fit_transform(X_train)

tfidf_mat = tfidf_mat.toarray()

tfidf_mat

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

classifierNB = GaussianNB()

classifierRF = RandomForestClassifier()

classifierDT = DecisionTreeClassifier()

classifierReg = LogisticRegression(max_iter=2000)

classifierNB.fit(tfidf_mat, y_train)
classifierRF.fit(tfidf_mat, y_train)
classifierDT.fit(tfidf_mat, y_train)
classifierReg.fit(tfidf_mat, y_train)

LogisticRegression(max_iter=2000)

In [20]:
# testing phase

test_mat = vectorizer.transform(X_test).toarray()

y_predNB = classifierNB.predict(test_mat)
y_predRF = classifierRF.predict(test_mat)
y_predDT = classifierDT.predict(test_mat)
y_predREG = classifierReg.predict(test_mat)

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix

accNB = accuracy_score(y_test, y_predNB)
accRF = accuracy_score(y_test, y_predRF)
accDT = accuracy_score(y_test, y_predDT)
accREG = accuracy_score(y_test, y_predREG)

matrix = confusion_matrix(y_predREG, y_test)

print("Accuracy Naive Bayes: ",accNB)
print("Accuracy Randon Forest:  ",accRF)
print("Accuracy Decision Tree:  ",accDT)
print("Accuracy Logistic Regression: ",accREG)



print("Confusion Matrix\n",matrix)

Accuracy Naive Bayes:  0.65189375
Accuracy Randon Forest:   0.655603125
Accuracy Decision Tree:   0.643609375
Accuracy Logistic Regression:  0.658903125
Confusion Matrix
 [[ 91231  40382]
 [ 68769 119618]]


In [30]:
'''import pickle
pickle.dump(vectorizer, open("vectoriser",'wb'))
pickle.dump(classifierReg, open("classifier",'wb'))'''

'import pickle\npickle.dump(vectorizer, open("vectoriser",\'wb\'))\npickle.dump(classifierReg, open("classifier",\'wb\'))'